In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras.layers import InputLayer, Dense, Conv2D, MaxPool2D, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from google.colab import drive

In [ ]:
# dataset, dataset_info = tfds.load(
#     "malaria", with_info=True, split=['train[:80%]','train[80%:90%]','train[90%:]'], shuffle_files=True
#     )
dataset, dataset_info = tfds.load(
    "malaria", with_info=True, shuffle_files=True
    )

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
dataset

In [ ]:
dataset_info

In [ ]:
def split(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO):
  DATASET_SIZE = len(dataset)

  train_dataset = dataset.take(int(TRAIN_RATIO * DATASET_SIZE))
  val_test_dataset = dataset.skip(int(TRAIN_RATIO * DATASET_SIZE))
  val_dataset = val_test_dataset.take(int(VAL_RATIO * DATASET_SIZE))
  test_dataset = val_test_dataset.skip(int(VAL_RATIO * DATASET_SIZE))

  return train_dataset, val_dataset, test_dataset

In [ ]:
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

train_ds, val_ds, test_ds = split(dataset["train"], TRAIN_RATIO, VAL_RATIO, TEST_RATIO)

In [ ]:
print(len(train_ds))
print(len(val_ds))
print( len(test_ds))

In [ ]:
def get_label_str(label):
  return "Parasitized" if label == 0 else "Uninfected"

In [ ]:
fig = plt.figure(figsize=(5, 5))
for i, sample in enumerate(train_ds.take(16)):
  ax = plt.subplot(4, 4, i+1)
  plt.imshow(sample['image'])
  plt.title(get_label_str(sample['label'].numpy()))
  plt.tight_layout()

In [ ]:
IM_size = 224
BATCH_SIZE = 32

In [ ]:
def resize_rescale(inputs):
  return tf.image.resize(inputs['image'], (IM_size, IM_size))/255., inputs['label']

In [ ]:
train_dataset = train_ds.map(resize_rescale).shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_ds.map(resize_rescale).shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_ds.map(resize_rescale).shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
for img, label in train_dataset.take(1):
  print(img, label)

In [ ]:
model = tf.keras.Sequential([
    InputLayer(input_shape=(IM_size, IM_size, 3)),
    Conv2D(filters = 6, kernel_size = 3, strides = 1, padding = 'valid', activation = 'relu'),
    BatchNormalization(),
    MaxPool2D(pool_size = 2, strides = 2),
    Conv2D(filters = 16, kernel_size = 3, strides = 1, padding = 'valid', activation = 'relu'),
    BatchNormalization(),
    MaxPool2D(pool_size = 2, strides = 2),

    Flatten(),
    Dense(100, activation = 'relu'),
    BatchNormalization(),
    Dense(10, activation = 'relu'),
    BatchNormalization(),
    Dense(1, activation = 'sigmoid')
])

model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.01), loss=BinaryCrossentropy(), metrics=BinaryAccuracy())

In [ ]:
history = model.fit(train_dataset, epochs=20, verbose=1, validation_data=val_dataset)

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['binary_accuracy'], label='train')
plt.plot(history.history['val_binary_accuracy'], label='val')
plt.title("model Accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend()
plt.show()

In [ ]:
model.evaluate(test_dataset)

In [ ]:
def parasite_or_not(x):
  if x < 0.5:
    return str("P")
  else:
    return str("U")

In [ ]:
parasite_or_not(model.predict(test_dataset.take(1))[0][0])

In [ ]:
for image, label in test_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3,3, i+1)
    plt.imshow(image[i])
    plt.title(parasite_or_not(label.numpy()[i]) + "-" + parasite_or_not(model.predict(image)[i][0]))
    plt.tight_layout()
    plt.axis('off')

In [ ]:
model.save("lenet")

In [ ]:
lenet_load_model = tf.keras.models.load_model("lenet")
lenet_load_model.summary()

In [ ]:
model.save("lenet.hdf5")

In [ ]:
lenet_load_model = tf.keras.models.load_model("lenet.hdf5")
lenet_load_model.summary()

In [ ]:
model.save_weight("weights/lenet_weights")

In [ ]:
model.load_weight("weights/lenet_weights")

In [ ]:
drive.mount('/content/drive/')

In [ ]:
!cp -r /content/lenet /content/drive/MyDrive/lenet_colab/

In [ ]:
!cp -r /content/drive/MyDrive/lenet_colab/ /content/lenet_colab/